<a href="https://colab.research.google.com/github/graphtrek/stockforecast/blob/main/graphtrek_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [323]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MAE
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import keras
from datetime import datetime, timedelta
import pandas as pd
import os, time
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MAE
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import keras
from datetime import datetime, timedelta
import pandas as pd
import os, time
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from keras.preprocessing.sequence import TimeseriesGenerator

In [324]:
ticker = "QQQ"
look_back = 14 #  number of past days we want to use to predict 1 day in the future.
max_data_size = 730 # ~2 years
split_percent = 0.90 # use 90% of the data  for train
INFO = 1
DEBUG = 2
TRACE = 3
print_level = DEBUG 
indicators_model_file_path = '/content/drive/MyDrive/models/' + ticker + '_' + str(look_back) + '_indicators_07.h5'
nr_of_indicators = 1
look_forward = 1
epochs = 200
learning_rate=0.0001
num_prediction = look_back * 3
np.random.seed(42)

In [325]:
rsi_url = 'https://www.alphavantage.co/query?function=RSI&datatype=json&symbol='+ticker+'&interval=daily&time_period='+str(look_back)+'&series_type=close&apikey=3F4URDEKOPLFH25T'
print(rsi_url)
rsi_api_response = pd.read_json(rsi_url)
#print(rsi_api_response)
rsi_api_data = rsi_api_response['Technical Analysis: RSI']
#print(rsi_api_data)
rsi_api_data = rsi_api_data.drop(
    index=['1: Symbol','2: Indicator','3: Last Refreshed','4: Interval','5: Time Period','6: Series Type','7: Time Zone']);
#print(rsi_api_data)

rsi_data = []
indicators_lstm_data = []
for key, value in rsi_api_data.items():
  date = key
  rsi = float(value.get('RSI'))
  rsi_data.append([date,rsi])
  indicators_lstm_data.append([rsi])

#print('rsi_data:', rsi_data)  
last_date =  str(rsi_data[0][0])

if max_data_size < len(rsi_data):
  rsi_data = np.flip(rsi_data[:max_data_size],axis=0)
  indicators_lstm_data = np.flip(indicators_lstm_data[:max_data_size],axis=0)
else:
  rsi_data = np.flip(rsi_data,axis=0)
  indicators_lstm_data = np.flip(indicators_lstm_data,axis=0)

first_rsi_data =  str(rsi_data[0])
last_rsi_data =  str(rsi_data[-1])
print('rsi_data_length:', len(rsi_data), 'first_rsi_data:', first_rsi_data, 'last_rsi_data:', last_rsi_data)  

https://www.alphavantage.co/query?function=RSI&datatype=json&symbol=QQQ&interval=daily&time_period=14&series_type=close&apikey=3F4URDEKOPLFH25T
rsi_data_length: 730 first_rsi_data: ['2018-12-20' '32.7186'] last_rsi_data: ['2021-11-11' '61.7091']


In [326]:
stock_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+ticker+'&outputsize=compact&apikey=3F4URDEKOPLFH25T'
print(stock_url)
stock_api_response = pd.read_json(stock_url)
stock_api_data = stock_api_response['Time Series (Daily)']
#print(stock_api_data)
stock_api_data = stock_api_data.drop(index=['1. Information','2. Symbol','3. Last Refreshed','4. Output Size','5. Time Zone']);

stock_data = []
split_coefficient = 1

for key, value in stock_api_data.items():
  date = key
  open = float(value.get('1. open'))
  high = float(value.get('2. high'))
  low = float(value.get('3. low'))
  close = float(value.get('4. close'))
  adjusted_close = float(value.get('5. adjusted close'))
  volume = int(value.get('6. volume'))
  divident = float(value.get('7. dividend amount'))

  if float(value.get('8. split coefficient')) > 1:
    split_coefficient = float(value.get('8. split coefficient'))
    
  open = open / split_coefficient
  high = high /split_coefficient
  close = close / split_coefficient
  low = low / split_coefficient
  
  stock_data.append([
      date,
      close,
      volume,
      high,
      low,
      open,
      divident
      ])

last_date =  str(stock_data[0][0])
print('data length:', len(stock_data), 'last_date:', last_date)

first_stock_data =  str(stock_data[0])
last_stock_data =  str(stock_data[-1])
print('stock_data_length:', len(stock_data), 'first_stock_data:', first_stock_data, 'last_stock_data:', last_stock_data)

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=QQQ&outputsize=compact&apikey=3F4URDEKOPLFH25T
data length: 100 last_date: 2021-11-11
stock_data_length: 100 first_stock_data: ['2021-11-11', 390.59, 32242983, 392.75, 390.24, 392.57, 0.0] last_stock_data: ['2021-06-23', 347.74, 27188454, 348.92, 346.88, 347.51, 0.0]


In [327]:
stock_df = pd.DataFrame(stock_data,columns=['Date','Close','Volume','High','Low','Open','Divident'])

In [328]:
indicators_df = pd.DataFrame(rsi_data,columns=['Date','RSI'])

split = int(split_percent*len(rsi_data))
if len(rsi_data) - split < look_back:
  split = look_back
split


indicators_df_train = indicators_df.iloc[:split]
indicators_df_test = indicators_df.iloc[split:]

indicators_lstm_train_data = indicators_lstm_data[:split]
indicators_lstm_test_data = indicators_lstm_data[split:]

In [329]:
np.set_printoptions(formatter={'float': '{: 0.12f}'.format})

indicators_scaler = MinMaxScaler(feature_range=(0, 1))
indicators_scaler = indicators_scaler.fit(indicators_lstm_data)

indicators_lstm_train_data_scaled = indicators_scaler.transform(indicators_lstm_train_data)
if print_level >= TRACE:
  print(indicators_lstm_train_data_scaled[:look_back+1])
  print(indicators_lstm_train_data_scaled.shape)

indicators_lstm_test_data_scaled = indicators_scaler.transform(indicators_lstm_test_data)
if print_level >= TRACE:
  print(indicators_lstm_test_data_scaled[:look_back+1])
  print(indicators_lstm_test_data_scaled.shape)

In [330]:
indicators_train_generator = TimeseriesGenerator(indicators_lstm_train_data_scaled, indicators_lstm_train_data_scaled, sampling_rate=1, length=look_back, batch_size=64)

if print_level >= TRACE:
  print('Samples: %d' % len(indicators_train_generator))
  # print each sample
  for i in range(2):
    x, y = indicators_train_generator[i]
    print('%s => %s' % (x, y))

In [331]:
indicators_model = None
try:
  indicators_model = keras.models.load_model(indicators_model_file_path)
  modified = os.path.getmtime(indicators_model_file_path)

  print(pd.to_datetime(last_date).date() - datetime.fromtimestamp(modified).date() )
  print('Loaded', indicators_model_file_path , ' model train date:',datetime.fromtimestamp(modified).date() , 'last date:', last_date)
except:
  indicators_model = None
  print('Model ' + indicators_model_file_path + ' does not exists.')

-1 day, 0:00:00
Loaded /content/drive/MyDrive/models/QQQ_14_indicators_07.h5  model train date: 2021-11-12 last date: 2021-11-11


In [332]:
if indicators_model is None:
  indicators_model = Sequential()
  indicators_model.add(LSTM(units=50, activation='tanh', input_shape=(look_back, nr_of_indicators), return_sequences=True))
  indicators_model.add(Dropout(0.2))
  indicators_model.add(LSTM(units=50, return_sequences=True, activation="tanh"))
  indicators_model.add(Dropout(0.2))
  indicators_model.add(LSTM(units=50, return_sequences=True, activation="tanh"))
  indicators_model.add(Dropout(0.2))
  indicators_model.add(LSTM(units=50, activation="tanh"))
  indicators_model.add(Dropout(0.2))
  indicators_model.add(Dense(nr_of_indicators))

  indicators_model.compile(
      loss=MAE,
      optimizer=Adam(learning_rate=learning_rate),
      metrics=["mae"]
  )
    
  #model.summary()
  print('Model ' + indicators_model_file_path + ' compiled.')

  indicators_modelo = indicators_model.fit(indicators_train_generator, epochs=epochs, verbose=0 )
  
  indicators_model.save(indicators_model_file_path)
  print('Saved model ' + indicators_model_file_path)

  if print_level >= DEBUG:
    plt.plot(indicators_modelo.history['loss'])
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

In [333]:
indicators_test_generator = TimeseriesGenerator(indicators_lstm_test_data_scaled, indicators_lstm_test_data_scaled,sampling_rate=1, length=look_back, batch_size=look_back)

print('Samples: %d' % len(indicators_test_generator))
# print each sample
if print_level >= TRACE:
  for i in range(2):
    x, y = indicators_test_generator[i]
    print('%s => %s' % (x, y))

Samples: 5


In [334]:
#Make prediction
indicators_prediction_scaled = indicators_model.predict(indicators_test_generator)

if print_level >= DEBUG:
  np.set_printoptions(formatter={'float': '{: 0.12f}'.format})
  print(indicators_prediction_scaled[:5])

indicators_prediction = indicators_scaler.inverse_transform(indicators_prediction_scaled)

if print_level >= DEBUG:
  np.set_printoptions(formatter={'float': '{: 0.2f}'.format})
  print('indicators_prediction shape:',indicators_prediction.shape)

  for x in range(look_back + 1):
    print('pred:',indicators_prediction[x], 'test:', indicators_lstm_test_data[x])


[[ 0.659786164761]
 [ 0.650177836418]
 [ 0.639579832554]
 [ 0.630900859833]
 [ 0.624921560287]]
indicators_prediction shape: (59, 1)
pred: [ 62.36] test: [ 60.85]
pred: [ 61.83] test: [ 63.95]
pred: [ 61.24] test: [ 64.68]
pred: [ 60.76] test: [ 67.75]
pred: [ 60.43] test: [ 63.61]
pred: [ 60.38] test: [ 64.59]
pred: [ 60.49] test: [ 59.78]
pred: [ 60.87] test: [ 58.23]
pred: [ 61.43] test: [ 60.54]
pred: [ 62.07] test: [ 62.72]
pred: [ 62.76] test: [ 62.99]
pred: [ 63.51] test: [ 54.31]
pred: [ 64.43] test: [ 46.78]
pred: [ 65.38] test: [ 50.46]
pred: [ 66.06] test: [ 57.30]


In [335]:
def predict(num_prediction, model):
    prediction_list = indicators_lstm_test_data_scaled[-look_back:]
    #print('prediction_list',prediction_list)
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        #print(x)
        x = x.reshape((1, look_back, nr_of_indicators))
        out = model.predict(x)
        #print(x,out) 
        prediction_list = np.append(prediction_list, out, axis=0)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(last_date,num_prediction):
    us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1,freq=us_bd).tolist()
    return prediction_dates

indicators_forecast_scaled = predict(num_prediction, indicators_model)
#print(forecast_scaled)
indicators_forecast_dates = predict_dates(indicators_df['Date'].values[-1],num_prediction)

indicators_forecast = indicators_scaler.inverse_transform(indicators_forecast_scaled.reshape((-1,nr_of_indicators)))
if nr_of_indicators == 1:
   indicators_forecast = indicators_forecast.reshape((-1))

print('indicators_forecast:',indicators_forecast)


indicators_forecast: [ 61.71  68.43  68.64  68.62  68.42  68.07  67.68  67.28  66.92  66.57
  66.24  65.99  65.81  65.74  65.82  65.80  65.64  65.49  65.35  65.22
  65.12  65.02  64.94  64.86  64.80  64.74  64.68  64.62  64.57  64.51
  64.45  64.41  64.36  64.32  64.28  64.25  64.21  64.18  64.15  64.12
  64.10  64.07  64.05]


In [336]:
trace1 = go.Candlestick(
    x = stock_df['Date'],
    open = stock_df['Open'],
    high = stock_df['High'],
    low = stock_df['Low'],
    close = stock_df['Close'],
    name = ticker + ' Price'
)


#'Date','Close','Volume','High','Low','Open','Divident'
open_price =  str(stock_data[-1][5])
low_price =  str(stock_data[-1][3])
high_price =  str(stock_data[-1][4])
close_price =  str(stock_data[-1][1])

layout = go.Layout(
    title = ticker + ' Date:' + last_date + ' Open:' + open_price + ' High:' + high_price + ' Low:' + low_price + ' Close:' + close_price,
    xaxis = {'title' : "Dates"},
    yaxis = {'title' : "Close Price ($)"},
    height = 450
)

fig = go.Figure(data=[trace1], layout=layout)

fig.update_yaxes(showspikes=True, spikemode='across', spikesnap='cursor',spikedash='dash')
fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor', spikedash='dash')
fig.update_layout(xaxis_rangeslider_visible=False)
config = dict({'scrollZoom': True})
fig.show(config=config)


trace_train = go.Scatter(
  x = indicators_df_train['Date'],
  y = indicators_df_train['RSI'],
  mode='lines',
  name ='RSI'
)

trace_test = go.Scatter(
  x = indicators_df_test['Date'],
  y = indicators_df_test['RSI'],
  mode='lines',
  name ='RSI'
)

trace_prediction = go.Scatter(
    x = indicators_df_test['Date'],
    y = indicators_prediction.reshape((-1)),
    name ='Test'
)

trace_forecast = go.Scatter(
    x = indicators_forecast_dates,
    y = indicators_forecast,
    mode='lines',
    name ='RSI Forecast'
)


indicators_layout = go.Layout(
    title = ticker + ' Date:' + last_date + ' Low RSI:' + min(indicators_df['RSI']) + ' High RSI:' + max(indicators_df['RSI']),
    xaxis = {'title' : "Dates"},
    yaxis = {'title' : "RSI"},
    height = 350
)

indicators_fig = go.Figure(data=[trace_train, trace_test,trace_prediction,trace_forecast], layout=indicators_layout)

indicators_fig.update_layout(shapes=[
dict(
  type= 'line',
  yref= 'y', y0= 30, y1= 30,   # adding a horizontal line at Y = 1
  xref= 'paper', x0= 0, x1= 1
  ) 
])

indicators_fig.update_yaxes(showspikes=True, spikemode='across', spikesnap='cursor',spikedash='dash')
indicators_fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor', spikedash='dash')
indicators_fig.update_layout(xaxis_rangeslider_visible=False)
config = dict({'scrollZoom': True})
indicators_fig.show(config=config)

indicators_fig.write_html('/content/drive/MyDrive/models/charts/'+ticker+ '_07_' + last_date + '_inidicators_forecast.html')

